## run this cell before starting spark cluster connection (corrects module error)

In [39]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.csv("/user/sprenner/lookup_clean_datasets.csv", sep="\t")

In [6]:
#df_lookup.limit(10).toPandas()

In [11]:
df_lookup_renamed = df_lookup.toDF('account','accountHash','clientState','clientStateHash','dataset','datasetHash',
                                   'datasetScope','datasetScopeHash','dataset_version','dataset_versionHash','datatype',
                                   'datatypeHash','eventType','eventTypeHash','eventVersion','eventVersionHash','filename',
                                   'filenameHash','filesize_double','hits','hostname','hostnameHash','prod_step','prod_stepHash',
                                   'project','projectHash','protocol','protocolHash','remoteSite','remoteSiteHash','run_number',
                                   'run_numberHash','scopeHash','stream_name','stream_nameHash','traceId','traceIdHash','traceIp',
                                   'traceIpHash','traceTimeentryUnix','transferStart','uuid','uuidHash','join_key_april','fileScope',
                                   'SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'BYTES', 'UPDATED_AT','CREATED_AT','join_key_lookup')

In [12]:
df_lookup_renamed.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   782618   782618                782618               
1  mean    None     1.9731572582537585E8  None                 
2  stddev  None     1.1370231297971807E9  None                 
3  min     aagaard  -1001038580           CannotAuthenticate   
4  max     zzheng   99410464              SourceNotFound       

        clientStateHash  \
0  782618                 
1  -9.74144880447121E8    
2  2.6189898366317806E8   
3  -1394468120            
4  634051254              

                                                                                               dataset  \
0  782618                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  782618                782618       782618                782618           
1  1.887588126440869E7   None         -7.555467303006039E8  None             
2  1.1947515338903446E9  None         8.327686484635457E8   None             
3  -1000582406           data10_hi    -1447489496           c1130_m1847      
4  99994791              mc16_valid   82887382              x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  782618                782618   782618                782618     
1  2.9641383972466004E8  None     4.950545094142839E8   None       
2  1.2265050444455745E9  None     1.1678571001883297E9  None       
3  -1001204191           AOD      -1048249511           download   
4  998683376             log      986936572             download   

  eventTypeHash eventVersion      eventVersionHash  \
0  782618        782618       782618                 
1  5.937808E7    None         -1.989902781594934E9   
2  0.0           None         3.146099197703217E8    
3  59378080      1.13.2       -153312916             
4  59378080      1.15.5       1261702107             

                             filename          filenameHash  \
0  782618                              782618                 
1  None                                -567794.2139920114     
2  None                                1.2403538392366464E9   
3  AOD.05371635._000001.pool.root.1    -1000005687            
4  log.13787275._000072.job.log.tgz.1  999998033              

        filesize_double               hits                  hostname  \
0  782618                782618             782618                     
1  6.143201947233235E8   2.870071989144129  None                       
2  1.2407176955045404E9  3.980098571762659  None                       
3  1.00000443E8          1                  ReCaS-ui-01.cs.infn.it     
4  999994.0              9                  xenia.nevis.columbia.edu   

           hostnameHash prod_step          prod_stepHash     project  \
0  782618                782618    782618                 782618       
1  -1.201348405637169E8  None      -2.7711860624417022E7  None         
2  1.2267734034224417E9  None      1.1064683221740832E9   None         
3  -1008250248           PhysCont  -1058503065            data10_hi    
4  999986343             simul     92057515               mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0  782618                782618   782618                782618                  
1  -7.553693460539573E8  None     3.5774461368790907E8  None                    
2  8.328094994956787E8   None     1.05261963242486E9    None                    
3  -1447489496           davs     -1364943180

In [13]:
df_lookup_renamed.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_lookup_renamed.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  0        0            0            0                0        0             

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  0             0                 0                0                     

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  0         0             0          0              0              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  0                 0         0             0                0     0          

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  0             0          0              0        0            0          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  0             0           0               0           0                

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0  0          0            0                0        0            0         

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  \
0  0            0                   0              0     0          

   join_key_april  fileScope  SCOPE  NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  \
0  0               0          0      0     0            0           0          

   CHILD_TYPE  BYTES  UPDATED_AT  CREATED_AT  join_key_lookup  
0  0           0      0           0           0

In [14]:
df_lookup_renamed.agg(*(countDistinct(col(c)).alias(c) for c in df_lookup_renamed.columns)).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  539      539          11           11               10574    10574         

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  23            23                3173             3173                  

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  112       112           1          1              5              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  5                 530096    530054        469129           38    344        

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  344           8          8              22       22           6          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  6             164         164             3434        3434             

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0  22         2954         2954             776980   776904       327       

   traceIpHash  traceTimeentryUnix  transferStart   uuid  uuidHash  \
0  327          776978              776978         12873  12873      

   join_key_april  fileScope  SCOPE   NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  \
0  530096          22         36     12704  22           530096      1          

   CHILD_TYPE   BYTES  UPDATED_AT  CREATED_AT  join_key_lookup  
0  1           469129  229218      433065      530096

In [35]:
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")
multi_datasets = spark.sql("SELECT *\
                      FROM (SELECT join_key_april, COUNT(DISTINCT NAME) AS distinct_datasets\
                      FROM lookup_renamed \
                      GROUP BY join_key_april \
                      ORDER BY distinct_datasets DESC)\
                  WHERE distinct_datasets > 1")
multi_datasets.toPandas()

join_key_april  distinct_datasets
0     mc16_13TeV:AOD.11264194._000001.pool.root.1         11               
1     mc16_13TeV:HITS.10701323._008539.pool.root.1        11               
2     mc16_13TeV:AOD.11264194._000022.pool.root.1         10               
3     mc16_13TeV:AOD.11264194._000004.pool.root.1         10               
4     mc16_13TeV:AOD.11264194._000032.pool.root.1         10               
5     mc16_13TeV:AOD.11264194._000031.pool.root.1         10               
6     mc16_13TeV:AOD.11264194._000058.pool.root.1         10               
7     mc16_13TeV:AOD.11264194._000015.pool.root.1         10               
8     mc16_13TeV:AOD.11264194._000008.pool.root.1         10               
9     mc16_13TeV:AOD.11264194._000043.pool.root.1         10               
10    mc16_13TeV:AOD.11264194._000089.pool.root.1         10               
11    mc16_13TeV:AOD.11264194._000075.pool.root.1         10               
12    mc16_13TeV:AOD.11264194._000072.pool.root.1         10               
13    mc16_13TeV:AOD.11264194._000085.pool.root.1         10               
14    mc16_13TeV:AOD.11264194._000029.pool.root.1         10               
15    mc16_13TeV:AOD.11264194._000025.pool.root.1         10               
16    mc16_13TeV:AOD.11264194._000060.pool.root.1         10               
17    mc16_13TeV:AOD.11264194._000036.pool.root.1         10               
18    mc16_13TeV:AOD.11264194._000076.pool.root.1         10               
19    mc16_13TeV:AOD.11264194._000037.pool.root.1         10               
20    mc16_13TeV:AOD.11264194._000069.pool.root.1         10               
21    mc16_13TeV:AOD.11264194._000057.pool.root.1         10               
22    mc16_13TeV:AOD.11264194._000044.pool.root.1         10               
23    mc16_13TeV:AOD.11264194._000046.pool.root.1         10               
24    mc16_13TeV:AOD.11264194._000027.pool.root.1         10               
25    mc16_13TeV:AOD.11264194._000073.pool.root.1         10               
26    mc16_13TeV:AOD.11264194._000013.pool.root.1         10               
27    mc16_13TeV:AOD.11264194._000088.pool.root.1         10               
28    mc16_13TeV:AOD.11264194._000012.pool.root.1         10               
29    mc16_13TeV:AOD.11264194._000054.pool.root.1         10               
...                                           ...         ..               
3110  mc16_13TeV:DAOD_SUSY5.13462844._000132.pool.root.1  2                
3111  mc15_valid:EVNT.13664314._000618.pool.root.1        2                
3112  mc15_valid:EVNT.13664314._000542.pool.root.1        2                
3113  mc16_13TeV:AOD.13642632._001201.pool.root.1         2                
3114  mc15_valid:EVNT.13664312._000681.pool.root.1        2                
3115  mc15_valid:EVNT.13637531._000895.pool.root.1        2                
3116  mc15_valid:EVNT.13785174._002080.pool.root.1        2                
3117  mc15_valid:EVNT.13664314._001018.pool.root.1        2                
3118  mc15_valid:EVNT.13664314._000292.pool.root.1        2                
3119  mc15_valid:EVNT.13664314._000679.pool.root.1        2                
3120  mc16_13TeV:DAOD_SUSY5.13462844._000165.pool.root.1  2                
3121  mc16_5p44TeV:AOD.13641049._000205.pool.root.1       2                
3122  mc15_valid:EVNT.13664312._000710.pool.root.1        2                
3123  mc16_13TeV:AOD.13642632._001209.pool.root.1         2                
3124  mc16_13TeV:AOD.13655447._000001.pool.root.1         2                
3125  mc15_14TeV:AOD.13709303._001887.pool.root.1         2                
3126  mc16_5p44TeV:AOD.13776843._000017.pool.root.1       2                
3127  mc15_valid:EVNT.13785144._000142.pool.root.1        2                
3128  mc16_5p44TeV:AOD.13641049._000200.pool.root.1       2                
3129  mc15_13TeV:EVNT.13705598._000053.pool.root.1        2                
3130  mc16_13TeV:DAOD_SUSY5.13463299._000081.pool.root.1  2                
3131  mc16_13T

In [38]:
multi_datasets.createOrReplaceTempView("multi_datasets")
single_datasets = spark.sql("SELECT *\
                             FROM lookup_renamed\
                             WHERE join_key_april NOT IN\
                             (SELECT join_key_april \
                              FROM multi_datasets)")
single_datasets.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   773511   773511                773511               
1  mean    None     1.9332918456033334E8  None                 
2  stddev  None     1.1368484730246296E9  None                 
3  min     aagaard  -1001038580           CannotAuthenticate   
4  max     zzheng   99410464              SourceNotFound       

        clientStateHash  \
0  773511                 
1  -9.742003650133947E8   
2  2.6245843422102356E8   
3  -1394468120            
4  634051254              

                                                                                               dataset  \
0  773511                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  773511                773511       773511                773511           
1  1.3454331428376583E7  None         -7.563716224422626E8  None             
2  1.1946892267743332E9  None         8.299273050637827E8   None             
3  -1000582406           data10_hi    -1447489496           c1130_m1847      
4  99994791              mc16_valid   82887382              x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  773511                773511   773511                773511     
1  2.9790405523572904E8  None     4.956404098287135E8   None       
2  1.2299931518516145E9  None     1.1714840899537017E9  None       
3  -1001204191           AOD      -1048249511           download   
4  998683376             log      986936572             download   

  eventTypeHash eventVersion       eventVersionHash  \
0  773511        773511       773511                  
1  5.937808E7    None         -1.9949883193320262E9   
2  0.0           None         2.999842806432518E8     
3  59378080      1.13.2       -153312916              
4  59378080      1.15.5       1261702107              

                             filename         filenameHash  \
0  773511                              773511                
1  None                                -199704.38392731326   
2  None                                1.240288437451886E9   
3  AOD.05371635._000001.pool.root.1    -1000005687           
4  log.13787275._000072.job.log.tgz.1  999998033             

        filesize_double                hits                  hostname  \
0  773511                773511              773511                     
1  6.114544373279915E8   2.8841257590389793  None                       
2  1.2418868943626182E9  3.999484813783933   None                       
3  1.00000443E8          1                   ReCaS-ui-01.cs.infn.it     
4  999994.0              9                   xenia.nevis.columbia.edu   

           hostnameHash prod_step         prod_stepHash     project  \
0  773511                773511    773511                773511       
1  -1.179659914208253E8  None      -3.703032686646214E7  None         
2  1.23018496307726E9    None      1.1032868216764987E9  None         
3  -1008250248           PhysCont  -1058503065           data10_hi    
4  999986343             simul     92057515              mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0  773511                773511   773511                773511                  
1  -7.562177887030708E8  None     3.552527886461123E8   None                    
2  8.299630089656088E8   None     1.0539434384134045E9  None                    
3  -1447489496           davs     -1364943180

In [18]:
df_lookup_renamed.select("dataset", "NAME").limit(10).toPandas()

dataset  \
0  data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402    
1  data15_13TeV.00284285.physics_Main.deriv.DAOD_EXOT5.r9264_p3083_p3481     
2  data15_13TeV.periodA.physics_Main.PhysCont.DAOD_EXOT8.grp15_v01_p3481     
3  data15_13TeV.periodA.physics_Main.PhysCont.DAOD_EXOT8.grp15_v01_p3481     
4  data15_13TeV.periodD.physics_Main.PhysCont.DAOD_EXOT8.grp15_v01_p3481     
5  data15_13TeV.periodD.physics_Main.PhysCont.DAOD_EXOT8.grp15_v01_p3481     
6  data15_13TeV.periodD.physics_Main.PhysCont.DAOD_HIGG3D1.grp15_v01_p3388   
7  data15_13TeV.periodD.physics_Main.PhysCont.DAOD_HIGG3D1.grp15_v01_p3388   
8  data15_13TeV.periodF.physics_Main.PhysCont.DAOD_HIGG3D1.grp15_v01_p3388   
9  data15_13TeV.periodF.physics_Main.PhysCont.DAOD_HIGG3D1.grp15_v01_p3388   

                                                                                     NAME  
0  data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
1  data15_13TeV.00284285.physics_Main.deriv.DAOD_EXOT5.r9264_p3083_p3481_tid13613132_00    
2  data15_13TeV.00267073.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466289_00    
3  data15_13TeV.00267162.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466294_00    
4  data15_13TeV.00276181.physics_Main.deriv.DAOD_EXOT8.r9412_p3083_p3481_tid13466382_00    
5  data15_13TeV.00276262.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466393_00    
6  data15_13TeV.00276329.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645178_00  
7  data15_13TeV.00276329.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645178_00  
8  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645374_00  
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645374_00

In [20]:
final = df_lookup_renamed.selectExpr('account','accountHash','clientState','clientStateHash','dataset as dataset_old','datasetHash as datasetHash_old',
                                   'datasetScope as datasetScope_old','datasetScopeHash as datasetScopeHash_old','dataset_version','dataset_versionHash','datatype',
                                   'datatypeHash','eventType','eventTypeHash','eventVersion','eventVersionHash','filename',
                                   'filenameHash','filesize_double','hits','hostname','hostnameHash','prod_step','prod_stepHash',
                                   'project','projectHash','protocol','protocolHash','remoteSite','remoteSiteHash','run_number',
                                   'run_numberHash','fileScope','scopeHash as fileScopeHash','stream_name','stream_nameHash','traceId','traceIdHash','traceIp',
                                   'traceIpHash','traceTimeentryUnix','transferStart','uuid','uuidHash',
                                   'SCOPE as datasetScope', 'NAME as dataset')
final.printSchema()

root
 |-- account: string (nullable = true)
 |-- accountHash: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- clientStateHash: string (nullable = true)
 |-- dataset_old: string (nullable = true)
 |-- datasetHash_old: string (nullable = true)
 |-- datasetScope_old: string (nullable = true)
 |-- datasetScopeHash_old: string (nullable = true)
 |-- dataset_version: string (nullable = true)
 |-- dataset_versionHash: string (nullable = true)
 |-- datatype: string (nullable = true)
 |-- datatypeHash: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventTypeHash: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- eventVersionHash: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filenameHash: string (nullable = true)
 |-- filesize_double: string (nullable = true)
 |-- hits: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- hostnameHash: string (nullable = true)
 |-- prod_step: string

In [26]:
final = final.withColumn("datasetHash", hash(final['dataset']))
final = final.withColumn("datasetScopeHash", hash(final['datasetScope']))

In [27]:
final.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   782618   782618                782618               
1  mean    None     1.9731572582537585E8  None                 
2  stddev  None     1.1370231297971807E9  None                 
3  min     aagaard  -1001038580           CannotAuthenticate   
4  max     zzheng   99410464              SourceNotFound       

       clientStateHash  \
0  782618                
1  -9.74144880447121E8   
2  2.618989836631781E8   
3  -1394468120           
4  634051254             

                                                                                           dataset_old  \
0  782618                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

        datasetHash_old datasetScope_old  datasetScopeHash_old  \
0  782618                782618           782618                 
1  1.887588126440869E7   None             -7.555467303006039E8   
2  1.1947515338903449E9  None             8.327686484635456E8    
3  -1000582406           data10_hi        -1447489496            
4  99994791              mc16_valid       82887382               

  dataset_version   dataset_versionHash datatype          datatypeHash  \
0  782618          782618                782618   782618                 
1  None            2.9641383972466004E8  None     4.950545094142839E8    
2  None            1.2265050444455745E9  None     1.1678571001883297E9   
3  c1130_m1847     -1001204191           AOD      -1048249511            
4  x557_m1947      998683376             log      986936572              

  eventType eventTypeHash eventVersion      eventVersionHash  \
0  782618    782618        782618       782618                 
1  None      5.937808E7    None         -1.989902781594934E9   
2  None      0.0           None         3.146099197703217E8    
3  download  59378080      1.13.2       -153312916             
4  download  59378080      1.15.5       1261702107             

                             filename          filenameHash  \
0  782618                              782618                 
1  None                                -567794.2139920114     
2  None                                1.2403538392366464E9   
3  AOD.05371635._000001.pool.root.1    -1000005687            
4  log.13787275._000072.job.log.tgz.1  999998033              

        filesize_double               hits                  hostname  \
0  782618                782618             782618                     
1  6.143201947233235E8   2.870071989144129  None                       
2  1.2407176955045407E9  3.980098571762659  None                       
3  1.00000443E8          1                  ReCaS-ui-01.cs.infn.it     
4  999994.0              9                  xenia.nevis.columbia.edu   

           hostnameHash prod_step          prod_stepHash     project  \
0  782618                782618    782618                 782618       
1  -1.201348405637169E8  None      -2.7711860624417022E7  None         
2  1.2267734034224417E9  None      1.1064683221740832E9   None         
3  -1008250248           PhysCont  -1058503065            data10_hi    
4  999986343             simul     92057515               mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0  782618                782618   782618                782618                  
1  -7.553693460539573E8  None     3.5774461368790907E8  None                    
2  8.328094994956787E8   None     1.0526196324248601E9  None                    
3  -1447489496           da

In [28]:
df_april = spark.read.json("/user/sprenner/training_official_april.json")
df_april.describe().toPandas()

summary  account          accountHash         clientState  \
0  count   777286   777286               777286               
1  mean    None     1.935684591732567E8  None                 
2  stddev  None     1.136131724108707E9  None                 
3  min     aagaard  -2127257360          CannotAuthenticate   
4  max     zzheng   2143592175           SourceNotFound       

        clientStateHash  \
0  777286                 
1  -9.74016245191591E8    
2  2.6212986830087787E8   
3  -1922357674            
4  1269532611             

                                                                                               dataset  \
0  777286                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  777286                777286       777286                777286           
1  1.4554549800768314E7  None         -7.552649492442358E8  None             
2  1.1941649925847797E9  None         8.3148655014496E8     None             
3  -2147401420           data10_hi    -2057006107           c1130_m1847      
4  2147445885            mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  777286                777286   777286                777286     
1  2.973310495739393E8   None     4.947060556405943E8   None       
2  1.2288188131163657E9  None     1.1700303292632797E9  None       
3  -2146716518           AOD      -2070622869           download   
4  2146405934            log      2124512252            download   

  eventTypeHash eventVersion      eventVersionHash  \
0  777286        777286       777286                 
1  5.937808E7    None         -1.992468944589716E9   
2  0.0           None         3.0732739647461045E8   
3  59378080      1.13.2       -2044360925            
4  59378080      1.15.5       1261702107             

                             filename         filenameHash  \
0  777286                              777286                
1  None                                -248557.46481861244   
2  None                                1.240265814971078E9   
3  AOD.05371635._000001.pool.root.1    -2147478094           
4  log.13787275._000072.job.log.tgz.1  2147482617            

        filesize_double                hits                  hostname  \
0  777286                777286              777286                     
1  6.11402028756827E8    2.8777927300890536  None                       
2  1.2408771809945776E9  3.991629368416235   None                       
3  304.0                 1                   ReCaS-ui-01.cs.infn.it     
4  1.1991446515E10       48                  xenia.nevis.columbia.edu   

            hostnameHash prod_step          prod_stepHash     project  \
0  777286                 777286    777286                 777286       
1  -1.1804844919942595E8  None      -3.3290311308177426E7  None         
2  1.2286452227051141E9   None      1.105007303744955E9    None         
3  -2133418459            PhysCont  -1329401808            data10_hi    
4  2129595856             simul     2037678380             mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0  777286                777286   777286                777286                  
1  -7.551860864380562E8  None     3.5590511037907284E8  None                    
2  8.315047181989845E8   None     1.0536208601928807E9  None                    
3  -2057006107           davs     -1607963002

In [29]:
final_dropped = final.dropDuplicates(['name', 'height'])

In [31]:
final_dropped.createOrReplaceTempView("final_dropped")

In [8]:
#final.write.format("json").save("/user/sprenner/training_official_april_cleaned_datasets.json")